In [1]:
import pandas as pd

In [2]:
tesla_stock = pd.read_csv("assets/TSLA.csv")
sc_shortage = pd.read_csv("assets/Semiconductor shortage affects.csv")
elon_tweets = pd.read_csv("assets/TweetsElonMusk.csv")

In [ ]:
df_news = pd.read_csv('AAPL_sentiment.csv')

In [3]:
elon_tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [4]:
tesla_tweets = elon_tweets[elon_tweets['tweet'].str.contains('tesla', case=False)]

new_dataframe = pd.DataFrame(tesla_tweets)

In [5]:
new_dataframe.tweet.head(50)


12     @tesla_adri @WholeMarsBlog These things are be...
26     @AustinTeslaClub @OwenSparks_ @WholeMarsBlog G...
28     @teslaownersSV @neuralink Turns out 🐒 love vid...
38                                       @TeslaGong Yeah
48     @IheartTesla Absolutely doable. Possibly as so...
52     @IheartTesla @neuralink Hopefully, later this ...
59     Thanks Tesla suppliers for providing us with c...
76     @Teslarati @ResidentSponge Special mention of ...
78     @Teslarati @ResidentSponge Great work by Tesla...
100                             @teslaownersSV @Tesla 🤣🤣
112    @OwenSparks_ Agreed! We’re not super far, as M...
136    @cleantechnica This is crazy. Should use Tesla...
145    @WholeMarsBlog I think there is a &gt;0% chanc...
163    Tesla is using only internal &amp; open source...
164                 You can now buy a Tesla with Bitcoin
187    @TeslaPhx @CodingMark @EvaFoxU @jrosinski97 @d...
194                  @Teslarati Преступленіе и наказаніе
201                            

In [6]:
import re

def preprocess_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)
    
    tweet = re.sub(r'@\w+|\#\w+', "", tweet)
    
    tweet = re.sub(r'[^\w\s]', "", tweet)
    
    tweet = tweet.lower()
    
    return tweet


In [9]:
tesla_tweets['processed_tweet'] = tesla_tweets['tweet'].apply(preprocess_tweet)

/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/1356004391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets['processed_tweet'] = tesla_tweets['tweet'].apply(preprocess_tweet)


In [10]:
tesla_tweets.processed_tweet

12         these things are best thought of as probabil...
26          good point   next major software rev will d...
28         turns out  love video games amp snacks just ...
38                                                    yeah
48        absolutely doable possibly as soon as neurali...
                               ...                        
12557        yeah not scalable my grad student idea was...
12558       that was my night job day job was working o...
12559       true ancient times  had to flip cpu registe...
12560                                           absolutely
12561      tesla is building up collision repair capabi...
Name: processed_tweet, Length: 2378, dtype: object

In [11]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive']

In [12]:
def analyze_sentiment(tweet):
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    
    output = model(**encoded_tweet)
    
    scores = output.logits.detach().numpy()[0]
    scores = np.exp(scores) / np.exp(scores).sum()
    
    max_score_index = np.argmax(scores)
    sentiment = labels[max_score_index]
    
    return sentiment

In [13]:
def analyze_sentiment2(tweet):
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    
    output = model(**encoded_tweet)
    
    scores = output.logits.detach().numpy()[0]
    scores = np.exp(scores) / np.exp(scores).sum()
    
    normalized_scores = scores - np.min(scores)
    normalized_scores /= np.max(normalized_scores)
    
    return normalized_scores


In [14]:
tesla_tweets['sentiment2'] = tesla_tweets['processed_tweet'].apply(analyze_sentiment2)

/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/2810372690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets['sentiment2'] = tesla_tweets['processed_tweet'].apply(analyze_sentiment2)


In [15]:
tesla_tweets.sentiment2

12        [0.0, 1.0, 0.8569846]
26       [0.0, 0.22127925, 1.0]
28       [0.0, 0.05958415, 1.0]
38       [0.0, 1.0, 0.30451855]
48       [0.0, 0.09361007, 1.0]
                  ...          
12557    [0.47573087, 1.0, 0.0]
12558    [0.0, 1.0, 0.09367881]
12559     [1.0, 0.7703952, 0.0]
12560    [0.0, 0.90690523, 1.0]
12561     [0.0, 0.5589807, 1.0]
Name: sentiment2, Length: 2378, dtype: object

In [16]:
tesla_tweets[labels] = pd.DataFrame(tesla_tweets['sentiment2'].tolist())

/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/1392690091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets[labels] = pd.DataFrame(tesla_tweets['sentiment2'].tolist())
/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/1392690091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets[labels] = pd.DataFrame(tesla_tweets['sentiment2'].tolist())
/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/1392690091.py:1: SettingWithCopyWarning: 
A v

In [18]:
tesla_tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'processed_tweet', 'sentiment2', 'Negative', 'Neutral',
       'Positive'],
      dtype='object')

In [19]:
import pandas as pd

tesla_tweets['date'] = pd.to_datetime(tesla_tweets['date'], format='%Y-%m-%d')

unique_dates_sentiment = tesla_tweets.groupby('date')['sentiment2'].mean()

new_dataframe = pd.DataFrame({'date': unique_dates_sentiment.index, 'average_sentiment': unique_dates_sentiment.values})

new_dataframe = new_dataframe.sort_values(by='date')

new_dataframe = new_dataframe.reset_index(drop=True)

print(new_dataframe)


          date                    average_sentiment
0   2011-12-31                [1.0, 0.7157263, 0.0]
1   2012-01-11                [0.0, 1.0, 0.6601804]
2   2012-01-13                [0.0, 0.4564618, 1.0]
3   2012-01-17                [1.0, 0.6496006, 0.0]
4   2012-01-18                [1.0, 0.4752943, 0.0]
..         ...                                  ...
931 2021-04-10         [0.0, 0.42695448, 0.9523282]
932 2021-04-13                [0.0, 0.5589807, 1.0]
933 2021-04-14  [0.32381523, 0.9354601, 0.21873577]
934 2021-04-15  [0.25481036, 0.80302113, 0.2172761]
935 2021-04-17              [0.0, 0.047333837, 1.0]

[936 rows x 2 columns]


/var/folders/yg/0l2nq7xx5yjbjdpkkfsnvb700000gn/T/ipykernel_67769/4151710855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets['date'] = pd.to_datetime(tesla_tweets['date'], format='%Y-%m-%d')


In [20]:
tweets_final = new_dataframe

In [22]:
tweets_final[labels] = pd.DataFrame(tweets_final['average_sentiment'].tolist())

In [23]:
tweets_final

,date,average_sentiment,Negative,Neutral,Positive
0,2011-12-31,"[1.0, 0.7157263, 0.0]",1.000000,0.715726,0.000000
1,2012-01-11,"[0.0, 1.0, 0.6601804]",0.000000,1.000000,0.660180
2,2012-01-13,"[0.0, 0.4564618, 1.0]",0.000000,0.456462,1.000000
3,2012-01-17,"[1.0, 0.6496006, 0.0]",1.000000,0.649601,0.000000
4,2012-01-18,"[1.0, 0.4752943, 0.0]",1.000000,0.475294,0.000000
...,...,...,...,...,...
931,2021-04-10,"[0.0, 0.42695448, 0.9523282]",0.000000,0.426954,0.952328
932,2021-04-13,"[0.0, 0.5589807, 1.0]",0.000000,0.558981,1.000000
933,2021-04-14,"[0.32381523, 0.9354601, 0.21873577]",0.323815,0.935460,0.218736
934,2021-04-15,"[0.25481036, 0.80302113, 0.2172761]",0.254810,0.803021,0.217276


In [24]:
tesla_stock

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-11-08,1.633333,1.666667,1.602000,1.665333,1.665333,7642500
1,2010-11-09,1.666667,1.712667,1.603333,1.642000,1.642000,14346000
2,2010-11-10,1.632000,1.998000,1.603333,1.957333,1.957333,45907500
3,2010-11-11,1.906667,1.940000,1.822000,1.869333,1.869333,29179500
4,2010-11-12,1.883333,2.033333,1.871333,1.989333,1.989333,40936500
...,...,...,...,...,...,...,...
3139,2023-05-01,163.169998,163.279999,158.830002,161.830002,161.830002,109015000
3140,2023-05-02,161.880005,165.490005,158.929993,160.309998,160.309998,128259700
3141,2023-05-03,160.009995,165.000000,159.910004,160.610001,160.610001,119728000
3142,2023-05-04,162.710007,162.949997,159.649994,161.199997,161.199997,95108500


In [25]:
import pandas as pd

tesla_stock['Date'] = pd.to_datetime(tesla_stock['Date'])

tesla_stock['Date'] += pd.DateOffset(days=3)

tesla_stock['Date'] = tesla_stock['Date'].dt.strftime('%Y-%m-%d')

print(tesla_stock)


            Date        Open        High         Low       Close   Adj Close   
0     2010-11-11    1.633333    1.666667    1.602000    1.665333    1.665333  \
1     2010-11-12    1.666667    1.712667    1.603333    1.642000    1.642000   
2     2010-11-13    1.632000    1.998000    1.603333    1.957333    1.957333   
3     2010-11-14    1.906667    1.940000    1.822000    1.869333    1.869333   
4     2010-11-15    1.883333    2.033333    1.871333    1.989333    1.989333   
...          ...         ...         ...         ...         ...         ...   
3139  2023-05-04  163.169998  163.279999  158.830002  161.830002  161.830002   
3140  2023-05-05  161.880005  165.490005  158.929993  160.309998  160.309998   
3141  2023-05-06  160.009995  165.000000  159.910004  160.610001  160.610001   
3142  2023-05-07  162.710007  162.949997  159.649994  161.199997  161.199997   
3143  2023-05-08  163.970001  170.789993  163.509995  170.059998  170.059998   

         Volume  
0       7642500  
1  

In [26]:
import pandas as pd

tweets_final['date'] = pd.to_datetime(tweets_final['date'], format='%Y-%m-%d')
tesla_stock['Date'] = pd.to_datetime(tesla_stock['Date'], format='%Y-%m-%d')

merged_data = pd.merge(tweets_final, tesla_stock, left_on='date', right_on='Date', how='inner')

print(merged_data)


          date                    average_sentiment  Negative   Neutral   
0   2011-12-31                [1.0, 0.7157263, 0.0]  1.000000  0.715726  \
1   2012-01-13                [0.0, 0.4564618, 1.0]  0.000000  0.456462   
2   2012-02-10  [0.33333334, 0.67837197, 0.3486577]  0.333333  0.678372   
3   2012-04-06               [0.0, 0.63404477, 1.0]  0.000000  0.634045   
4   2012-05-10               [0.12222295, 1.0, 0.0]  0.122223  1.000000   
..         ...                                  ...       ...       ...   
630 2021-04-08               [0.0, 0.13217683, 1.0]  0.000000  0.132177   
631 2021-04-09         [0.0, 0.57106835, 0.7681729]  0.000000  0.571068   
632 2021-04-10         [0.0, 0.42695448, 0.9523282]  0.000000  0.426954   
633 2021-04-15  [0.25481036, 0.80302113, 0.2172761]  0.254810  0.803021   
634 2021-04-17              [0.0, 0.047333837, 1.0]  0.000000  0.047334   

     Positive       Date        Open        High         Low       Close   
0    0.000000 2011-12-3

In [33]:
final_merged_tweets = merged_data.drop(columns = ["Date", "average_sentiment"])

In [50]:
merged_data

,date,average_sentiment,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-12-31,-1.000000,2011-12-31,1.932667,1.949333,1.869333,1.900667,1.900667,8628000
1,2012-01-13,1.000000,2012-01-13,1.829333,1.850667,1.816667,1.841333,1.841333,10077000
2,2012-02-10,0.015324,2012-02-10,2.120000,2.120000,2.054667,2.106667,2.106667,15324000
3,2012-04-06,1.000000,2012-04-06,2.446667,2.564667,2.444667,2.534000,2.534000,16471500
4,2012-05-10,-0.122223,2012-05-10,2.130667,2.172000,2.107333,2.164667,2.164667,17370000
...,...,...,...,...,...,...,...,...,...
630,2021-04-08,1.000000,2021-04-08,235.903336,236.053329,228.233337,230.350006,230.350006,125528400
631,2021-04-09,0.768173,2021-04-09,230.100006,232.183334,227.123337,230.539993,230.539993,84815400
632,2021-04-10,0.952328,2021-04-10,229.000000,230.460007,222.613327,223.656662,223.656662,78928200
633,2021-04-15,-0.037534,2021-04-15,228.566666,234.933334,227.363327,233.993332,233.993332,87407100


In [34]:
final_merged_tweets.to_csv("tesla_sentiment.csv")

# added prediction based on elon tweet data

In [35]:
tesla_deaths = pd.read_csv("assets/Tesla Deaths - Sudden Acceleration.csv")

In [44]:
tesla_deaths['Incident Date']

<bound method Series.unique of 0        22-January-15
1         2-October-19
2      29-September-19
3      24-September-19
4        18-October-19
            ...       
117          29-Nov-17
118          24-Feb-16
119           4-Aug-15
120          21-Sep-13
121          29-Jul-13
Name: Incident Date, Length: 122, dtype: object>

In [42]:
final_merged_tweets.date

0     2011-12-31
1     2012-01-13
2     2012-02-10
3     2012-04-06
4     2012-05-10
         ...    
630   2021-04-08
631   2021-04-09
632   2021-04-10
633   2021-04-15
634   2021-04-17
Name: date, Length: 635, dtype: datetime64[ns]

In [ ]:
converted_dates = pd.to_datetime(tesla_deaths['Incident Date'], format='%d-%B-%y').dt.strftime('%Y-%m-%d')

# Print the converted dates
print(converted_dates)

In [58]:
month_mapping = {
    'January': '01', 'Jan': '01', 'Jan.': '01',
    'February': '02', 'Feb': '02', 'Feb.': '02',
    'March': '03', 'Mar': '03', 'Mar.': '03',
    'April': '04', 'Apr': '04', 'Apr.': '04',
    'May': '05',
    'June': '06', 'Jun': '06', 'Jun.': '06',
    'July': '07', 'Jul': '07', 'Jul.': '07',
    'August': '08', 'Aug': '08', 'Aug.': '08',
    'September': '09', 'Sept': '09', 'Sept.': '09','Sep': '09',
    'October': '10', 'Oct': '10', 'Oct.': '10', '10ober':'10',
    'November': '11', 'Nov': '11', 'Nov.': '11',
    'December': '12', 'Dec': '12', 'Dec.': '12'
}

In [61]:
tesla_deaths['Incident Date'] = tesla_deaths['Incident Date'].replace(month_mapping, regex=True).apply(pd.to_datetime, format='%d-%m-%y').dt.strftime('%Y-%m-%d')

In [63]:
tesla_deaths['Incident Date']

0      2015-01-22
1      2019-10-02
2      2019-09-29
3      2019-09-24
4      2019-10-18
          ...    
117    2017-11-29
118    2016-02-24
119    2015-08-04
120    2013-09-21
121    2013-07-29
Name: Incident Date, Length: 122, dtype: object

In [64]:
final_merged_tweets

,date,Negative,Neutral,Positive,Open,High,Low,Close,Adj Close,Volume
0,2011-12-31,1.000000,0.715726,0.000000,1.932667,1.949333,1.869333,1.900667,1.900667,8628000
1,2012-01-13,0.000000,0.456462,1.000000,1.829333,1.850667,1.816667,1.841333,1.841333,10077000
2,2012-02-10,0.333333,0.678372,0.348658,2.120000,2.120000,2.054667,2.106667,2.106667,15324000
3,2012-04-06,0.000000,0.634045,1.000000,2.446667,2.564667,2.444667,2.534000,2.534000,16471500
4,2012-05-10,0.122223,1.000000,0.000000,2.130667,2.172000,2.107333,2.164667,2.164667,17370000
...,...,...,...,...,...,...,...,...,...,...
630,2021-04-08,0.000000,0.132177,1.000000,235.903336,236.053329,228.233337,230.350006,230.350006,125528400
631,2021-04-09,0.000000,0.571068,0.768173,230.100006,232.183334,227.123337,230.539993,230.539993,84815400
632,2021-04-10,0.000000,0.426954,0.952328,229.000000,230.460007,222.613327,223.656662,223.656662,78928200
633,2021-04-15,0.254810,0.803021,0.217276,228.566666,234.933334,227.363327,233.993332,233.993332,87407100
